# Greeks Analysis and Portfolio Risk Management

This notebook covers:
1. Understanding option Greeks (Delta, Gamma, Vega, Theta, Rho)
2. Calculating Greeks for individual options
3. Portfolio Greeks aggregation
4. Greeks profiles across strikes and time
5. Risk management and hedging strategies
6. Practical trading applications

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_delta,
    black_scholes_gamma,
    black_scholes_vega,
    black_scholes_theta,
)
from options_desk.core.option import OptionType

## 1. Understanding the Greeks

### Delta (Δ)
- **Measures:** Sensitivity of option price to $1 change in underlying
- **Range:** 0 to 1 for calls, -1 to 0 for puts
- **Interpretation:** Delta = 0.5 means if stock moves $1, option moves $0.50
- **Use:** Hedging ratio (buy/sell delta units of stock to hedge)

### Gamma (Γ)
- **Measures:** Rate of change of Delta
- **Interpretation:** How much delta will change for $1 move in underlying
- **Highest:** At-the-money options
- **Use:** Measure of hedging risk

### Vega (ν)
- **Measures:** Sensitivity to 1% change in implied volatility
- **Always positive:** for both calls and puts
- **Highest:** At-the-money options with longer time to expiry
- **Use:** Volatility trading

### Theta (Θ)
- **Measures:** Time decay - price change per day
- **Usually negative:** Options lose value as time passes
- **Highest:** At-the-money options near expiry
- **Use:** Measure cost of carrying options

### Rho (ρ)
- **Measures:** Sensitivity to 1% change in risk-free rate
- **Less important:** for short-term options
- **Positive:** for calls, negative for puts

## 2. Fetch Market Data

In [ ]:
# Fetch SPY data
TICKER = 'SPY'
ticker = yf.Ticker(TICKER)

# Get current price
stock_data = ticker.history(period='5d')
spot_price = stock_data['Close'].iloc[-1]

print(f"Ticker: {TICKER}")
print(f"Current Spot Price: ${spot_price:.2f}")

# Select an expiration (30-45 days out is typical)
expiration_dates = ticker.options
selected_expiry = expiration_dates[5]  # Adjust index as needed

# Fetch option chain
opt_chain = ticker.option_chain(selected_expiry)
calls = opt_chain.calls
puts = opt_chain.puts

# Calculate time to expiry
expiry_datetime = pd.to_datetime(selected_expiry)
days_to_expiry = (expiry_datetime - pd.Timestamp.now()).days
time_to_expiry = days_to_expiry / 365.25

print(f"\nSelected Expiry: {selected_expiry}")
print(f"Days to Expiry: {days_to_expiry}")
print(f"Time to Expiry (years): {time_to_expiry:.4f}")

## 3. Calculate Greeks for ATM Option

In [ ]:
# Find ATM strike
atm_strike = calls.iloc[(calls['strike'] - spot_price).abs().argsort()[:1]]['strike'].values[0]
atm_call = calls[calls['strike'] == atm_strike].iloc[0]
atm_put = puts[puts['strike'] == atm_strike].iloc[0]

# Use market implied volatility
iv_call = atm_call['impliedVolatility']
iv_put = atm_put['impliedVolatility']
avg_iv = (iv_call + iv_put) / 2

risk_free_rate = 0.045

print(f"\n{'='*70}")
print(f"ATM Option Analysis - Strike: ${atm_strike:.2f}")
print(f"{'='*70}")

# Calculate Greeks for CALL
print(f"\n{'='*30} CALL {'='*30}")
call_price = black_scholes_price(spot_price, atm_strike, time_to_expiry, 
                                  risk_free_rate, avg_iv, OptionType.CALL)
call_delta = black_scholes_delta(spot_price, atm_strike, time_to_expiry, 
                                  risk_free_rate, avg_iv, OptionType.CALL)
call_gamma = black_scholes_gamma(spot_price, atm_strike, time_to_expiry, 
                                  risk_free_rate, avg_iv)
call_vega = black_scholes_vega(spot_price, atm_strike, time_to_expiry, 
                                risk_free_rate, avg_iv)
call_theta = black_scholes_theta(spot_price, atm_strike, time_to_expiry, 
                                  risk_free_rate, avg_iv, OptionType.CALL)

print(f"Theoretical Price: ${call_price:.2f}")
print(f"Market Price: ${atm_call['lastPrice']:.2f}")
print(f"\nGreeks:")
print(f"  Delta:   {call_delta:>8.4f}  (option moves ${abs(call_delta):.4f} per $1 underlying move)")
print(f"  Gamma:   {call_gamma:>8.4f}  (delta changes by {call_gamma:.4f} per $1 underlying move)")
print(f"  Vega:    {call_vega:>8.4f}  (option moves ${call_vega:.4f} per 1% IV change)")
print(f"  Theta:   {call_theta:>8.4f}  (option loses ${abs(call_theta):.4f} per day)")

# Calculate Greeks for PUT
print(f"\n{'='*30} PUT {'='*30}")
put_price = black_scholes_price(spot_price, atm_strike, time_to_expiry, 
                                 risk_free_rate, avg_iv, OptionType.PUT)
put_delta = black_scholes_delta(spot_price, atm_strike, time_to_expiry, 
                                 risk_free_rate, avg_iv, OptionType.PUT)
put_vega = black_scholes_vega(spot_price, atm_strike, time_to_expiry, 
                               risk_free_rate, avg_iv)
put_theta = black_scholes_theta(spot_price, atm_strike, time_to_expiry, 
                                 risk_free_rate, avg_iv, OptionType.PUT)

print(f"Theoretical Price: ${put_price:.2f}")
print(f"Market Price: ${atm_put['lastPrice']:.2f}")
print(f"\nGreeks:")
print(f"  Delta:   {put_delta:>8.4f}  (option moves ${abs(put_delta):.4f} per $1 underlying move)")
print(f"  Gamma:   {call_gamma:>8.4f}  (same as call)")
print(f"  Vega:    {put_vega:>8.4f}  (same as call)")
print(f"  Theta:   {put_theta:>8.4f}  (option loses ${abs(put_theta):.4f} per day)")

## 4. Greeks Profiles Across Strikes

In [ ]:
# Calculate Greeks for all strikes
strikes = calls['strike'].values

greeks_data = []

for strike in strikes:
    # Get IV from market
    call_row = calls[calls['strike'] == strike]
    put_row = puts[puts['strike'] == strike]
    
    if len(call_row) > 0 and len(put_row) > 0:
        iv = (call_row['impliedVolatility'].iloc[0] + put_row['impliedVolatility'].iloc[0]) / 2
        
        # Calculate Greeks
        call_delta = black_scholes_delta(spot_price, strike, time_to_expiry, 
                                         risk_free_rate, iv, OptionType.CALL)
        put_delta = black_scholes_delta(spot_price, strike, time_to_expiry, 
                                        risk_free_rate, iv, OptionType.PUT)
        gamma = black_scholes_gamma(spot_price, strike, time_to_expiry, 
                                    risk_free_rate, iv)
        vega = black_scholes_vega(spot_price, strike, time_to_expiry, 
                                  risk_free_rate, iv)
        call_theta = black_scholes_theta(spot_price, strike, time_to_expiry, 
                                         risk_free_rate, iv, OptionType.CALL)
        put_theta = black_scholes_theta(spot_price, strike, time_to_expiry, 
                                        risk_free_rate, iv, OptionType.PUT)
        
        greeks_data.append({
            'strike': strike,
            'moneyness': strike / spot_price,
            'call_delta': call_delta,
            'put_delta': put_delta,
            'gamma': gamma,
            'vega': vega,
            'call_theta': call_theta,
            'put_theta': put_theta
        })

greeks_df = pd.DataFrame(greeks_data)

print(f"Greeks calculated for {len(greeks_df)} strikes")
greeks_df.head(10)

## 5. Visualize Greeks Profiles

In [ ]:
# Create comprehensive Greeks visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Delta
axes[0, 0].plot(greeks_df['strike'], greeks_df['call_delta'], 'g-', linewidth=2, label='Call Delta')
axes[0, 0].plot(greeks_df['strike'], greeks_df['put_delta'], 'r-', linewidth=2, label='Put Delta')
axes[0, 0].axvline(spot_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[0, 0].axhline(0, color='gray', linestyle='-', linewidth=0.5)
axes[0, 0].axhline(0.5, color='blue', linestyle=':', linewidth=1, alpha=0.5)
axes[0, 0].axhline(-0.5, color='blue', linestyle=':', linewidth=1, alpha=0.5)
axes[0, 0].set_xlabel('Strike Price')
axes[0, 0].set_ylabel('Delta')
axes[0, 0].set_title('Delta Profile', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Gamma
axes[0, 1].plot(greeks_df['strike'], greeks_df['gamma'], 'b-', linewidth=2)
axes[0, 1].axvline(spot_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[0, 1].fill_between(greeks_df['strike'], greeks_df['gamma'], alpha=0.3)
axes[0, 1].set_xlabel('Strike Price')
axes[0, 1].set_ylabel('Gamma')
axes[0, 1].set_title('Gamma Profile', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Vega
axes[0, 2].plot(greeks_df['strike'], greeks_df['vega'], 'purple', linewidth=2)
axes[0, 2].axvline(spot_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[0, 2].fill_between(greeks_df['strike'], greeks_df['vega'], alpha=0.3, color='purple')
axes[0, 2].set_xlabel('Strike Price')
axes[0, 2].set_ylabel('Vega')
axes[0, 2].set_title('Vega Profile', fontweight='bold')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# Theta
axes[1, 0].plot(greeks_df['strike'], greeks_df['call_theta'], 'g-', linewidth=2, label='Call Theta')
axes[1, 0].plot(greeks_df['strike'], greeks_df['put_theta'], 'r-', linewidth=2, label='Put Theta')
axes[1, 0].axvline(spot_price, color='black', linestyle='--', linewidth=1.5, label='Spot')
axes[1, 0].axhline(0, color='gray', linestyle='-', linewidth=0.5)
axes[1, 0].set_xlabel('Strike Price')
axes[1, 0].set_ylabel('Theta (per day)')
axes[1, 0].set_title('Theta Profile', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Moneyness view - Delta
axes[1, 1].plot(greeks_df['moneyness'], greeks_df['call_delta'], 'g-', linewidth=2, label='Call Delta')
axes[1, 1].plot(greeks_df['moneyness'], greeks_df['put_delta'], 'r-', linewidth=2, label='Put Delta')
axes[1, 1].axvline(1.0, color='black', linestyle='--', linewidth=1.5, label='ATM')
axes[1, 1].axhline(0, color='gray', linestyle='-', linewidth=0.5)
axes[1, 1].set_xlabel('Moneyness (K/S)')
axes[1, 1].set_ylabel('Delta')
axes[1, 1].set_title('Delta vs Moneyness', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Moneyness view - Gamma
axes[1, 2].plot(greeks_df['moneyness'], greeks_df['gamma'], 'b-', linewidth=2)
axes[1, 2].axvline(1.0, color='black', linestyle='--', linewidth=1.5, label='ATM')
axes[1, 2].fill_between(greeks_df['moneyness'], greeks_df['gamma'], alpha=0.3)
axes[1, 2].set_xlabel('Moneyness (K/S)')
axes[1, 2].set_ylabel('Gamma')
axes[1, 2].set_title('Gamma vs Moneyness', fontweight='bold')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Greeks Evolution with Time to Expiry

Analyze how Greeks change as option approaches expiration.

In [ ]:
# Simulate Greeks decay over time for ATM option
days_until_expiry = np.arange(1, 91)  # 90 days
times_to_expiry = days_until_expiry / 365.25

# Storage
time_decay_data = []

for days, time_years in zip(days_until_expiry, times_to_expiry):
    delta = black_scholes_delta(spot_price, atm_strike, time_years, 
                                risk_free_rate, avg_iv, OptionType.CALL)
    gamma = black_scholes_gamma(spot_price, atm_strike, time_years, 
                                risk_free_rate, avg_iv)
    vega = black_scholes_vega(spot_price, atm_strike, time_years, 
                              risk_free_rate, avg_iv)
    theta = black_scholes_theta(spot_price, atm_strike, time_years, 
                                risk_free_rate, avg_iv, OptionType.CALL)
    
    time_decay_data.append({
        'days': days,
        'delta': delta,
        'gamma': gamma,
        'vega': vega,
        'theta': theta
    })

decay_df = pd.DataFrame(time_decay_data)

# Plot Greeks evolution
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Delta evolution
axes[0, 0].plot(decay_df['days'], decay_df['delta'], 'g-', linewidth=2)
axes[0, 0].set_xlabel('Days to Expiry')
axes[0, 0].set_ylabel('Delta')
axes[0, 0].set_title('ATM Call Delta Evolution', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].invert_xaxis()

# Gamma evolution
axes[0, 1].plot(decay_df['days'], decay_df['gamma'], 'b-', linewidth=2)
axes[0, 1].set_xlabel('Days to Expiry')
axes[0, 1].set_ylabel('Gamma')
axes[0, 1].set_title('ATM Gamma Evolution (increases near expiry)', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].invert_xaxis()

# Vega evolution
axes[1, 0].plot(decay_df['days'], decay_df['vega'], color='purple', linewidth=2)
axes[1, 0].set_xlabel('Days to Expiry')
axes[1, 0].set_ylabel('Vega')
axes[1, 0].set_title('ATM Vega Evolution (decreases near expiry)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].invert_xaxis()

# Theta evolution
axes[1, 1].plot(decay_df['days'], decay_df['theta'], 'r-', linewidth=2)
axes[1, 1].set_xlabel('Days to Expiry')
axes[1, 1].set_ylabel('Theta (per day)')
axes[1, 1].set_title('ATM Theta Evolution (accelerates near expiry)', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].invert_xaxis()

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print(f"- Delta: Relatively stable around 0.5 for ATM options")
print(f"- Gamma: Explodes as expiry approaches (max: {decay_df['gamma'].max():.4f})")
print(f"- Vega: Decreases as expiry approaches (long-dated options more sensitive to IV)")
print(f"- Theta: Accelerates near expiry (time decay speeds up)")

## 7. Portfolio Greeks

Calculate aggregate Greeks for a portfolio of options.

In [ ]:
# Example portfolio: Iron Condor
# Buy 1 Put at 95% of spot
# Sell 1 Put at 97.5% of spot
# Sell 1 Call at 102.5% of spot
# Buy 1 Call at 105% of spot

portfolio = [
    {'position': 'LONG', 'type': 'PUT', 'strike': spot_price * 0.95, 'quantity': 1},
    {'position': 'SHORT', 'type': 'PUT', 'strike': spot_price * 0.975, 'quantity': 1},
    {'position': 'SHORT', 'type': 'CALL', 'strike': spot_price * 1.025, 'quantity': 1},
    {'position': 'LONG', 'type': 'CALL', 'strike': spot_price * 1.05, 'quantity': 1},
]

# Calculate portfolio Greeks
portfolio_greeks = {
    'delta': 0,
    'gamma': 0,
    'vega': 0,
    'theta': 0
}

print("\n" + "="*70)
print("IRON CONDOR PORTFOLIO ANALYSIS")
print("="*70)
print(f"\n{'Position':<8} {'Type':<6} {'Strike':<10} {'Qty':<5} {'Delta':<10} {'Gamma':<10} {'Vega':<10} {'Theta':<10}")
print("-" * 85)

for leg in portfolio:
    option_type = OptionType.CALL if leg['type'] == 'CALL' else OptionType.PUT
    quantity = leg['quantity'] if leg['position'] == 'LONG' else -leg['quantity']
    strike = leg['strike']
    
    # Calculate Greeks
    delta = black_scholes_delta(spot_price, strike, time_to_expiry, 
                                risk_free_rate, avg_iv, option_type) * quantity
    gamma = black_scholes_gamma(spot_price, strike, time_to_expiry, 
                                risk_free_rate, avg_iv) * quantity
    vega = black_scholes_vega(spot_price, strike, time_to_expiry, 
                              risk_free_rate, avg_iv) * quantity
    theta = black_scholes_theta(spot_price, strike, time_to_expiry, 
                                risk_free_rate, avg_iv, option_type) * quantity
    
    # Aggregate
    portfolio_greeks['delta'] += delta
    portfolio_greeks['gamma'] += gamma
    portfolio_greeks['vega'] += vega
    portfolio_greeks['theta'] += theta
    
    print(f"{leg['position']:<8} {leg['type']:<6} ${strike:<9.2f} {quantity:>4} "
          f"{delta:>9.4f} {gamma:>9.4f} {vega:>9.4f} {theta:>9.4f}")

print("-" * 85)
print(f"{'TOTAL':<25} {'':<10} "
      f"{portfolio_greeks['delta']:>9.4f} {portfolio_greeks['gamma']:>9.4f} "
      f"{portfolio_greeks['vega']:>9.4f} {portfolio_greeks['theta']:>9.4f}")

print("\n" + "="*70)
print("PORTFOLIO RISK SUMMARY")
print("="*70)
print(f"Delta: {portfolio_greeks['delta']:>8.4f}  → {'Delta neutral' if abs(portfolio_greeks['delta']) < 0.1 else 'Not delta neutral'}")
print(f"Gamma: {portfolio_greeks['gamma']:>8.4f}  → {'Negative gamma (seller)' if portfolio_greeks['gamma'] < 0 else 'Positive gamma (buyer)'}")
print(f"Vega:  {portfolio_greeks['vega']:>8.4f}  → {'Short volatility' if portfolio_greeks['vega'] < 0 else 'Long volatility'}")
print(f"Theta: {portfolio_greeks['theta']:>8.4f}  → {'Earning ${:.2f}/day'.format(portfolio_greeks['theta']) if portfolio_greeks['theta'] > 0 else 'Losing ${:.2f}/day'.format(abs(portfolio_greeks['theta']))}")

## 8. P&L Simulation with Greeks

Simulate portfolio P&L for different market scenarios.

In [ ]:
# Simulate P&L for spot price changes
spot_range = np.linspace(spot_price * 0.90, spot_price * 1.10, 100)

# Calculate P&L using Greeks (first-order approximation)
pnl_linear = []
pnl_with_gamma = []

for s in spot_range:
    delta_s = s - spot_price
    
    # Linear approximation (Delta only)
    pnl_delta = portfolio_greeks['delta'] * delta_s * 100  # *100 for per-contract basis
    pnl_linear.append(pnl_delta)
    
    # With Gamma correction
    pnl_gamma = pnl_delta + 0.5 * portfolio_greeks['gamma'] * (delta_s ** 2) * 100
    pnl_with_gamma.append(pnl_gamma)

# Plot P&L profile
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(spot_range, pnl_linear, '--', linewidth=2, label='P&L (Delta only)', alpha=0.7)
ax.plot(spot_range, pnl_with_gamma, '-', linewidth=2, label='P&L (Delta + Gamma)', color='red')
ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.axvline(spot_price, color='green', linestyle='--', linewidth=2, label='Current Spot')

# Mark the strikes
for leg in portfolio:
    ax.axvline(leg['strike'], color='gray', linestyle=':', linewidth=1, alpha=0.5)

ax.set_xlabel('Spot Price', fontsize=12)
ax.set_ylabel('P&L ($)', fontsize=12)
ax.set_title('Iron Condor P&L Profile', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Iron Condor profits when underlying stays within the short strikes")
print("- Max profit at current spot (theta decay)")
print("- Losses accelerate outside the range due to negative gamma")
print(f"- Theta earning: ${portfolio_greeks['theta']:.2f} per day")

## 9. Hedging with Greeks

### Delta Hedging Example

In [ ]:
# Example: Long 10 ATM calls, need to delta hedge
num_calls = 10
call_strike = atm_strike

# Calculate delta of position
position_delta = black_scholes_delta(spot_price, call_strike, time_to_expiry, 
                                     risk_free_rate, avg_iv, OptionType.CALL) * num_calls

# Hedge ratio: short this many shares
hedge_shares = -position_delta * 100  # Each option is 100 shares

print("\n" + "="*70)
print("DELTA HEDGING EXAMPLE")
print("="*70)
print(f"\nPosition: LONG {num_calls} ATM Calls @ ${call_strike:.2f}")
print(f"\nCall Delta: {position_delta/num_calls:.4f} per contract")
print(f"Total Position Delta: {position_delta:.4f}")
print(f"\nHedge Required: {'SHORT' if hedge_shares < 0 else 'LONG'} {abs(hedge_shares):.0f} shares of {TICKER}")
print(f"\nAfter hedging:")
print(f"  Net Delta: ~0 (delta neutral)")
print(f"  P&L now driven by: Gamma (curvature), Vega (volatility), Theta (time decay)")

# Simulate P&L with and without hedge
spot_scenarios = np.linspace(spot_price * 0.95, spot_price * 1.05, 50)

pnl_unhedged = []
pnl_hedged = []

for s in spot_scenarios:
    # Options P&L
    new_call_price = black_scholes_price(s, call_strike, time_to_expiry, 
                                         risk_free_rate, avg_iv, OptionType.CALL)
    original_call_price = black_scholes_price(spot_price, call_strike, time_to_expiry, 
                                              risk_free_rate, avg_iv, OptionType.CALL)
    option_pnl = (new_call_price - original_call_price) * num_calls * 100
    
    # Stock hedge P&L
    stock_pnl = hedge_shares * (s - spot_price)
    
    pnl_unhedged.append(option_pnl)
    pnl_hedged.append(option_pnl + stock_pnl)

# Plot
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(spot_scenarios, pnl_unhedged, 'r-', linewidth=2, label='Unhedged (Long Calls only)')
ax.plot(spot_scenarios, pnl_hedged, 'g-', linewidth=2, label='Delta Hedged (Calls + Short Stock)')
ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.axvline(spot_price, color='blue', linestyle='--', linewidth=2, label='Current Spot')

ax.set_xlabel('Spot Price', fontsize=12)
ax.set_ylabel('P&L ($)', fontsize=12)
ax.set_title('Delta Hedging: Unhedged vs Hedged Position', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("- Unhedged: P&L increases linearly with spot (directional bet)")
print("- Delta Hedged: P&L relatively flat for small moves (isolated from direction)")
print("- Hedged position still has gamma risk (curvature) and vega risk (volatility)")

## 10. Summary

### Key Takeaways:

1. **Delta** - Directional exposure
   - Use for hedging directional risk
   - ~0.5 for ATM options

2. **Gamma** - Rate of delta change
   - Peaks at ATM
   - Explodes near expiry
   - Negative gamma = selling options = need frequent rehedging

3. **Vega** - Volatility exposure
   - Higher for long-dated options
   - Peaks at ATM
   - Buy options = long vega = profit from IV increase

4. **Theta** - Time decay
   - Accelerates near expiry
   - Short options = positive theta = earn time decay
   - Long options = negative theta = pay for time

5. **Portfolio Greeks** - Aggregate risk metrics
   - Calculate net exposure across all positions
   - Use for position sizing and risk limits

### Next Notebook:
**04_delta_hedging.ipynb** - Simulate dynamic delta hedging strategies with realistic transaction costs and rebalancing.